In [2]:
import os
from numpy.random import choice
import faker
import pandas as pd
from tqdm.notebook import trange

The dataset is available in the [Google Drive](https://drive.google.com/file/d/1gviMipwq5MM_0O_gE02PsTq0-2tNGfbo/view?usp=sharing). Download it into the `data` folder.

In [5]:
n = 10_000_000

dr = f'data'
f = f'fake_data_people_{n}.parquet'

filename = os.path.join(dr, f)
if f not in os.listdir(dr):
    fake = faker.Faker(seed=0, include_optional=[])
    data = []
    for i in trange(n):
        data.append({
            'name': fake.name() + choice(["", " " + fake.language_name()], p=[0.4, 0.6]),
            'email': fake.email(),
            'address': fake.address()
        })
    df = pd.DataFrame(data)
    df.to_parquet(filename, index=False)

df = pd.read_parquet(filename)
print(f"Number of duplicate names: {df['name'].value_counts().loc[lambda x: x > 1].shape[0] / df.shape[0] * 100:.2f}%")
df

Number of duplicate names: 5.53%


,name,email,address
0,Phillip Ray,eric97@example.com,"5035 Mathis Squares Suite 555\nEast Jenna, LA ..."
1,Austin Martinez,ramosthomas@example.net,"1994 Emily Union Apt. 780\nWest Kaylashire, ND..."
2,Caitlin Williams,alexisbridges@example.net,"0873 Richard Curve\nAdamsfurt, AK 91206"
3,David Kline,nortonyvonne@example.net,"120 Huber Isle\nAntoniomouth, GA 03771"
4,Steven Phillips,sbarry@example.net,"547 Donna Viaduct Apt. 585\nNew Rubenview, GU ..."
...,...,...,...
9999995,Gregory Nguyen,umacdonald@example.org,"1080 Burke Groves Apt. 339\nKyleberg, UT 50077"
9999996,Ashley Fowler,sandra51@example.com,"4812 Jason Junction Suite 359\nLake Kimberly, ..."
9999997,David Clark,collierlisa@example.net,"86943 Dave Road Suite 219\nSmithmouth, AR 56196"
9999998,Robert Foster,christian09@example.org,"59594 Harper Field\nNorth Jason, TN 87310"


In [8]:
print(f"Total duplicated names: {df['name'].value_counts().loc[lambda x: x > 1].shape[0]:,}")

Total duplicated names: 553,473


In [9]:
print(f"Total duplicated emails: {df['email'].value_counts().loc[lambda x: x > 1].shape[0]:,}")

Total duplicated emails: 1,061,303


In [11]:
non_unic_emails = df['email'].value_counts().loc[lambda x: x > 1].index
non_unic_emails

Index(['ssmith@example.com', 'hsmith@example.org', 'nsmith@example.com',
       'dsmith@example.com', 'rsmith@example.org', 'psmith@example.com',
       'csmith@example.org', 'ysmith@example.net', 'nsmith@example.org',
       'ismith@example.org',
       ...
       'nicholsbrooke@example.net', 'phillipsanders@example.net',
       'hayleyjohnson@example.net', 'reedluis@example.com',
       'kimphilip@example.net', 'hamiltonadrian@example.com',
       'loricoleman@example.net', 'davisvickie@example.org',
       'carolynknapp@example.net', 'fisherkelli@example.org'],
      dtype='object', name='email', length=1061303)

In [13]:
import recordlinkage
time_start = pd.Timestamp.now()
rules = [
    {"field": "name", "type": "Exact", "weight": 50},
    {"field": "email", "type": "Exact", "weight": 20},
    {"field": "address", "type": "Fuzzy", "weight": 30}
]

comp = recordlinkage.Compare()

comp.exact('name', 'name', agree_value=50)
comp.exact('email', 'email', agree_value=20)
comp.string('address', 'address', method='levenshtein', threshold=0.85)

indexer = recordlinkage.Index()
indexer.block(left_on='name', right_on='name')
indexer.block(left_on='email', right_on='email')
candidate_pairs = indexer.index(df.sample(frac=0.1))
time_end = pd.Timestamp.now()
print(f"Time to compute candidate pairs: {time_end - time_start}")
print(f"Number of candidate pairs: {candidate_pairs.shape[0]:,}")

Time to compute candidate pairs: 0 days 00:00:29.282876
Number of candidate pairs: 10,008,243


In [14]:
time_start = pd.Timestamp.now()
computed = comp.compute(candidate_pairs, df)
computed.columns = ['name_score', 'email_score', 'address_score']
computed['address_score_converted'] = computed['address_score'] * rules[2]['weight']
computed['score'] = computed[['name_score', 'email_score', 'address_score_converted']].sum(axis=1)
computed = computed.sort_values(by='score', ascending=False)
computed = computed.reset_index()
time_end = pd.Timestamp.now()
print(f"Time to compute scores: {time_end - time_start}")
print(f"Number of computed pairs: {computed.shape[0]:,}")

Time to compute scores: 0 days 00:02:46.291974
Number of computed pairs: 10,008,243


Prepare the final report

In [15]:
computed_grouped = (computed.groupby(['level_0', 'score'])
                    .agg(cluster_size = ('level_1', 'count'))
                    .reset_index())
computed_grouped.columns = ['Cluster Id', 'Score', 'Cluster size']
computed_grouped = computed_grouped[['Cluster Id', 'Cluster size', 'Score']]
computed_grouped

,Cluster Id,Cluster size,Score
0,5,13,20.0
1,5,4,50.0
2,10,3,50.0
3,18,13,50.0
4,66,4,20.0
...,...,...,...
1148963,9999942,4,20.0
1148964,9999942,7,50.0
1148965,9999973,2,50.0
1148966,9999988,3,50.0


### What to do next? 

There is severa strategies to consider. 

Strategy 1: For the cases when there is much more the 5-10% of duplicated names, or other exact columns, we can consider to prefilter the daraset to remove the duplicated records before the linkage process.

Strategy 2: Split dataset into **sorted** blocks and compare records only within the same block. This will reduce the number of comparisons and speed up the process. This strategy also adds the possibility to scale the process.

<br>

### Extreme cases

For the cases with huge datasets, we can consider to use approximate algorithms. These algorithms are able to find the similar records with high probability, but they are not able to find the exact matches. But this is the topic for another discussion.